In [1]:
from keras.datasets import cifar10
import keras

In [2]:
# 超參數設定
LEARNING_RATE = 1e-3
EPOCHS = 10
BATCH_SIZE = 32
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 將資料攤平成一維資料
# x_train = x_train.reshape(50000, 3072) 
# x_test = x_test.reshape(10000, 3072)

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
x_test shape: (10000, 32, 32, 3)
50000 train samples
10000 test samples


In [4]:
def cifar_generator(image_array, batch_size=32):
    while True:
        for indexs in range(0, len(image_array), batch_size):
            images = x_train[indexs: indexs+batch_size]
            labels = y_train[indexs: indexs+batch_size]
            yield images, labels

In [5]:
cifar_gen = cifar_generator(x_train, BATCH_SIZE)

## 作業

請參考昨天的程式碼，將訓練資料讀取方式改寫成 Generator，並將原本的 model.fit 改為 model.fit_generator 來進行訓練。請參考 Keras [官方文件中 fit_generator 的說明](https://keras.io/models/sequential/)

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam

In [7]:
# model build
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 64)        1

In [8]:
model.fit_generator(cifar_gen,
                    steps_per_epoch=len(x_train)//BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1,
                    validation_data=(x_test, y_test))
#batch_size=BATCH_SIZE,Do not specify the validation_batch_size if your data is in the form of datasets, generators
#shuffle=True

score = model.evaluate(x_test, y_test, verbose=0)

print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
1562/1562 [==============================] - 32s 21ms/step - loss: 1.5468 - accuracy: 0.4429 - val_loss: 1.2558 - val_accuracy: 0.5541
Epoch 2/10
1562/1562 [==============================] - 31s 20ms/step - loss: 1.1630 - accuracy: 0.5940 - val_loss: 0.9266 - val_accuracy: 0.6736
Epoch 3/10
1562/1562 [==============================] - 31s 20ms/step - loss: 1.0296 - accuracy: 0.6452 - val_loss: 0.8841 - val_accuracy: 0.6929
Epoch 4/10
1562/1562 [==============================] - 32s 20ms/step - loss: 0.9850 - accuracy: 0.6642 - val_loss: 0.8331 - val_accuracy: 0.7107
Epoch 5/10
1562/1562 [==============================] - 33s 21ms/step - loss: 0.9815 - accuracy: 0.6715 - val_loss: 0.9123 - val_accuracy: 0.6884
Epoch 6/10
1562/1562 [==============================] - 33s 21ms/step - loss: 0.9857 - accuracy: 0.6723 - val_loss: 1.1166 - val_accuracy: 0.6470
Epoch 7/10
1562/1562 [==========================